# Assignment 2
### Machine Learning
### Robert Knox

## Imports

In [59]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
import os
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
import time
from operator import itemgetter


In [2]:
with open('bottle.csv') as f:
    line = f.readline()
line_column_names = line.split(',')

desired_cols = ['T_degC','Salnty','STheta']
df = pd.read_csv('bottle.csv',sep=',',header='infer',index_col=None,usecols=desired_cols)

df.head()

,T_degC,Salnty,STheta
0,10.50,33.440,25.649
1,10.46,33.440,25.656
2,10.46,33.437,25.654
3,10.45,33.420,25.643
4,10.45,33.421,25.643


In [6]:
df.dropna(axis=0,inplace=True)
df.isna().any()

T_degC    False
Salnty    False
STheta    False
dtype: bool

In [7]:
#Get values from dataframe
T_degC = df['T_degC'].values
Salnty = df['Salnty'].values
STheta = df['STheta'].values

#put the predictors into a corresponding X matrix
X = np.column_stack((Salnty,STheta))

#Build train & test using sklearn's train_test_split function
X_train, X_test, y_train, y_test = train_test_split(X, T_degC, test_size=0.25, random_state=42)

#scale the training data
scaler_train = StandardScaler(copy=False,with_mean=True, with_std=True)
scaler_train.fit(X_train)
X_train_scaled = scaler_train.transform(X_train)
#scale the testing data using teh same scaler as train
X_test_scaled=scaler_train.transform(X_test)

#add in the intercepts
X_train_b0 = np.ones(len(X_train_scaled))
X_test_b0 = np.ones(len(X_test_scaled))
X_train_scaled = np.column_stack((X_train_b0,X_train_scaled))
X_test_scaled = np.column_stack((X_test_b0,X_test_scaled))

In [10]:
#deal with list/array issue
y_test = np.reshape(y_test,(len(y_test),1))
y_train = np.reshape(y_train,(len(y_train),1))

#Check that arrays are properly shaped
print(y_test.shape,'\n',y_train.shape,'\n',X_test_scaled.shape,'\n',X_train_scaled.shape)

(203044, 1) 
 (609130, 1) 
 (203044, 3) 
 (609130, 3)


## Mini-Batch Gradient Descent

In [132]:


def minibatch_grad_descent(minibatch_size):
    np.random.seed(123)
    
    #begin timing analysis
    t0 = time.clock()
    m = len(y_train)
    epochs = 100
    theta = np.random.randn(3,1)  # random initialization of 3 values
    
    #instantiate predicted y's out here so we can reuse in function
    ypred = X_train_scaled.dot(theta)
    cost_array = np.zeros(epochs)
    gradient_list =[]

    for epoch in range(epochs):
        cost = 0.0
        #shuffle indices so we start with a random set of data
        shuffled_indices = np.random.permutation(m)

        #get the rows for my shuffled index for both X & y
        X_b_shuffled = X_train_scaled[shuffled_indices]
        y_shuffled = y_train[shuffled_indices]

        #perform batch gradient descent on & do it in chunks of mini-batch size
        for i in range(0, m, minibatch_size):
            
            xi = X_b_shuffled[i:i+minibatch_size]
            yi = y_shuffled[i:i+minibatch_size]
            pred = xi.dot(theta)
            gradients = (2/minibatch_size) * xi.T.dot(xi.dot(theta) - yi)
            
            eta = 0.1
            theta = theta - eta * gradients
            gradient_list.append(gradients)
            
        
        #calculate the predictions using our theta from running the minibatch
        ypred = X_train_scaled.dot(theta)
        MSE = mean_squared_error(y_train,ypred)
        cost_array[epoch]=MSE
    
    #we want to find the gradient_list that performed the best, aka minimized MSE
    #Ending time of analysis
    t1 = time.clock()
    #summarize theta
    print('Mini-Batch size:{}\tEpochs:{}'.format(minibatch_size,epochs))
    print('Analysis time:',np.round(t1-t0,4),' seconds')
    print('Theta:'+str(theta))
    #predict y using X_test_scaled & theta from the loop
    ypredtest = X_test_scaled.dot(theta)
    #use metrics modules compare yhat to y_test
    #RMSE
    MSE = mean_squared_error(y_test,ypredtest)
    print('MSE:',MSE)
    #R2Score
    R2score = r2_score(y_test,ypredtest)
    print('R2score:',R2score)
    #Explained Variance
    ExpVar = explained_variance_score(y_test,ypredtest)
    print('Explained Variance:',ExpVar)
    

In [133]:
minibatch_grad_descent(50)

Mini-Batch size:50	Epochs:100
Analysis time: 22.2021  seconds
Theta:[[10.8226751 ]
 [ 1.43597329]
 [-6.00573282]]
MSE: 4.882304945209515
R2score: 0.72553602318795
Explained Variance: 0.7255630970772766


In [134]:
minibatch_grad_descent(2000)

Mini-Batch size:2000	Epochs:100
Analysis time: 12.2394  seconds
Theta:[[10.84108761]
 [ 1.25789214]
 [-5.76964934]]
MSE: 4.5455453350885735
R2score: 0.7444673236414702
Explained Variance: 0.7444681691963568


In [135]:
minibatch_grad_descent(10000)

Mini-Batch size:10000	Epochs:100
Analysis time: 11.616  seconds
Theta:[[10.8414851 ]
 [-0.73599002]
 [-3.17573544]]
MSE: 4.668360672002829
R2score: 0.7375631285612196
Explained Variance: 0.7375661421850817


## Check against linear regression

In [131]:
lin_reg = LinearRegression()
reg = lin_reg.fit(X_train_scaled, y_train)
print("Intercept: ",reg.intercept_,"\nThetas: ", reg.coef_)
print("R-Squared:\t\t{0:.2f}".format(reg.score(X_test_scaled,y_test)))
yhat_sk = reg.predict(X_test_scaled)
print("MSE:\t\t\t{0:.2f}".format(mean_squared_error(y_test,yhat_sk)))
print('Explained Variance:\t{0:.2f}'.format(explained_variance_score(y_test,yhat_sk)))

Intercept:  [10.84736466] 
Thetas:  [[ 0.         -0.15561296 -3.3378831 ]]
R-Squared:		0.76
MSE:			4.25
Explained Variance:	0.76


## Conclusion

I was able to achieve good results compared to the linear regression. The MSE & R squared are similar to those found using ordinary least squares regression. The size of the mini-batch affects the overall performance speed since it drives the total number of iterations through the interior for loop.

Future improvements to the code would be to switch to a while loop instead of a for loop, monitoring the percent change in the MSE. Currently, eta is a constant and could instead be modeled as a learning schedule. I would also incorporate a max iterations check to prevent the function from running continuously.